### 编程实践二 理解MRP、MDP以及相关知识

![马尔科夫奖励过程示例](https://github.com/kirsguo/kirsguo_library/blob/master/Code_RL/RL_kirsguo/2.MDP/mrp.png)

In [1]:
import numpy as np

#### 下面代码定义学生马尔科夫奖励过程所需要的信息
#### 其中状态集数量为7（两个字典双向映射），状态转移概率用7*7矩阵表示
#### 折扣因子为gamma 奖励函数用reward表示，分别与状态对应

In [2]:
# num_states = 7
# {"0": "C1", "1":"C2", "2":"C3", "3":"Pass", "4":"Pub", "5":"FB", "6":"Sleep"}
i_to_n = {}
i_to_n["0"] = "C1"
i_to_n["1"] = "C2"
i_to_n["2"] = "C3"
i_to_n["3"] = "Pass"
i_to_n["4"] = "Pub"
i_to_n["5"] = "FB"
i_to_n["6"] = "Sleep"

n_to_i = {}
for i, name in zip(i_to_n.keys(), i_to_n.values()):
    n_to_i[name] = int(i)
    
#   C1   C2   C3  Pass  Pub  FB  Sleep
Pss = [
   [0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0],
   [0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.2],
   [0.0, 0.0, 0.0, 0.6, 0.4, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
   [0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0],
   [0.1, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
]
Pss = np.array(Pss)
rewards = [-2, -2, -2, 10, 1, -1, 0]
gamma = 0.5


#### 定义以状态C1开始的马尔科夫链

In [3]:
chains = [
    ["C1", "C2", "C3", "Pass", "Sleep"],
    ["C1", "FB", "FB", "C1", "C2", "Sleep"],
    ["C1", "C2", "C3", "Pub", "C2", "C3", "Pass", "Sleep"],
    ["C1", "FB", "FB", "C1", "C2", "C3", "Pub", "C1", "FB",
     "FB", "FB", "C1", "C2", "C3", "Pub", "C2", "Sleep"]
]

In [4]:


def compute_return(start_index=0, 
                   chain=None, 
                   gamma=0.5) -> float:
    '''
    计算一个马尔科夫奖励过程中某状态的收获值
    :param start_index: 要计算的状态在链中的位置
    :param chain: 要计算的马尔科夫过程
    :param gamma: 折扣因子
    :return: 收获
    '''
    count, power,  = 0.0, 0
    for i in range(start_index, len(chain)):
        count += np.power(gamma, power) * rewards[n_to_i[chain[i]]]
        power += 1
    return count

#### 验证上文定义最后一条马尔科夫链的起始状态收获值

In [5]:
compute_return(0, chains[3], gamma=0.5)

-3.196044921875

In [6]:
def compute_value(Pss, rewards, gamma = 0.05):
    '''
    通过求解矩阵方程的形式直接计算状态的价值
    :param Pss: 状态转移概率矩阵 shape(7, 7)
    :param rewards: 即时奖励 list
    :param gamma: 折扣因子
    :return: values 各状态的价值
    '''
    
    rewards = np.array(rewards).reshape((-1, 1))
    values = np.dot(np.linalg.inv(np.eye(7, 7) - gamma * Pss), rewards)
    return values

#### 求解状态价值

In [7]:
values = compute_value(Pss, rewards, gamma = 0.999999)
print(values)

[[-12.54296219]
 [  1.4568013 ]
 [  4.32100594]
 [ 10.        ]
 [  0.80253065]
 [-22.54274676]
 [  0.        ]]


![马尔科夫决策过程示例](https://github.com/kirsguo/kirsguo_library/blob/master/Code_RL/RL_kirsguo/2.MDP/mdp.png)

#### 首先我们先定义一些基本的操作，我们将基于行为的状态转移概率矩阵以及基于行为的奖励函数用字典来存储

In [1]:
# %load ./Jupyter_Notebook_workplace/utils.py
# 辅助函数
def str_key(*args):
    '''将参数用"_"连接起来作为字典的键，需注意参数本身可能会是tuple或者list型，
    比如类似((a,b,c),d)的形式。
    '''
    new_arg = []
    for arg in args:
        if type(arg) in [tuple, list]:
            new_arg += [str(i) for i in arg]
        else:
            new_arg.append(str(arg))
    return "_".join(new_arg)


def set_dict(target_dict, value, *args):
    target_dict[str_key(*args)] = value


def set_prob(P, s, a, s1, p=1.0):  # 设置概率字典,默认为1
    set_dict(P, p, s, a, s1)


def get_prob(P, s, a, s1):  # 获取概率值
    return P.get(str_key(s, a, s1), 0)


def set_reward(R, s, a, r):  # 设置奖励字典
    set_dict(R, r, s, a)


def get_reward(R, s, a):  # 获取奖励值
    return R.get(str_key(s, a), 0)


def display_dict(target_dict):  # 显示字典内容
    for key in target_dict.keys():
        print("{}:　{:.2f}".format(key, target_dict[key]))
    print("")


# 辅助方法
def set_value(V, s, v):  # 设置价值字典
    set_dict(V, v, s)


def get_value(V, s):  # 获取价值值
    return V.get(str_key(s), 0)


def set_pi(Pi, s, a, p=0.5):  # 设置策略字典
    set_dict(Pi, p, s, a)


def get_pi(Pi, s, a):  # 获取策略（概率）值
    return Pi.get(str_key(s, a), 0)

In [2]:

# 构建学生马尔科夫决策过程
S = ['浏览手机中', '第一节课', '第二节课', '第三节课', '休息中']
A = ['浏览手机', '学习', '离开浏览', '泡吧', '退出学习']
R = {} # 基于状态和行为的奖励Rsa
P = {} # 基于行为的状态转移概率Pss'a
gamma = 1.0 # 衰减因子

set_prob(P, S[0], A[0], S[0])  # 浏览手机中 - 浏览手机 -> 浏览手机中
set_prob(P, S[0], A[2], S[1])  # 浏览手机中 - 离开浏览 -> 第一节课
set_prob(P, S[1], A[0], S[0])  # 第一节课 - 浏览手机 -> 浏览手机中
set_prob(P, S[1], A[1], S[2])  # 第一节课 - 学习 -> 第二节课
set_prob(P, S[2], A[1], S[3])  # 第二节课 - 学习 -> 第三节课
set_prob(P, S[2], A[4], S[4])  # 第二节课 - 退出学习 -> 退出休息
set_prob(P, S[3], A[1], S[4])  # 第三节课 - 学习 -> 退出休息
set_prob(P, S[3], A[3], S[1], p=0.2)  # 第三节课 - 泡吧 -> 第一节课
set_prob(P, S[3], A[3], S[2], p=0.4)  # 第三节课 - 泡吧 -> 第一节课
set_prob(P, S[3], A[3], S[3], p=0.4)  # 第三节课 - 泡吧 -> 第一节课

set_reward(R, S[0], A[0], -1)  # 浏览手机中 - 浏览手机 -> -1
set_reward(R, S[0], A[2],  0)  # 浏览手机中 - 离开浏览 -> 0
set_reward(R, S[1], A[0], -1)  # 第一节课 - 浏览手机 -> -1
set_reward(R, S[1], A[1], -2)  # 第一节课 - 学习 -> -2
set_reward(R, S[2], A[1], -2)  # 第二节课 - 学习 -> -2
set_reward(R, S[2], A[4],  0)  # 第二节课 - 退出学习 -> 0
set_reward(R, S[3], A[1], 10)  # 第三节课 - 学习 -> 10
set_reward(R, S[3], A[3], +1)  # 第三节课 - 泡吧 -> -1

MDP = (S, A, R, P, gamma)

#### 以上就是我们通过代码初始的一个马尔科夫决策过程的模型，接下来我们可以调用display_dict方法来验证一下

In [3]:
print("----基于行为的状态转移概率字典（矩阵）信息:----")
display_dict(P)
print("----基于状态和行为奖励字典（函数）信息:----")
display_dict(R)

----基于行为的状态转移概率字典（矩阵）信息:----
浏览手机中_浏览手机_浏览手机中:　1.00
浏览手机中_离开浏览_第一节课:　1.00
第一节课_浏览手机_浏览手机中:　1.00
第一节课_学习_第二节课:　1.00
第二节课_学习_第三节课:　1.00
第二节课_退出学习_休息中:　1.00
第三节课_学习_休息中:　1.00
第三节课_泡吧_第一节课:　0.20
第三节课_泡吧_第二节课:　0.40
第三节课_泡吧_第三节课:　0.40

----基于状态和行为奖励字典（函数）信息:----
浏览手机中_浏览手机:　-1.00
浏览手机中_离开浏览:　0.00
第一节课_浏览手机:　-1.00
第一节课_学习:　-2.00
第二节课_学习:　-2.00
第二节课_退出学习:　0.00
第三节课_学习:　10.00
第三节课_泡吧:　1.00



#### 接下来我们将进行策略评估，在这里我们采用均一随机策略，将某一状态下采取所有行为的可能性概率相等。这里用Pi来表示策略

In [4]:
# S = ['浏览手机中','第一节课','第二节课','第三节课','休息中']
# A = ['继续浏览','学习','离开浏览','泡吧','退出学习']
# 设置行为策略：pi(a|.) = 0.5
Pi = {}
set_pi(Pi, S[0], A[0], 0.5) # 浏览手机中 - 浏览手机
set_pi(Pi, S[0], A[2], 0.5) # 浏览手机中 - 离开浏览
set_pi(Pi, S[1], A[0], 0.5) # 第一节课 - 浏览手机
set_pi(Pi, S[1], A[1], 0.5) # 第一节课 - 学习
set_pi(Pi, S[2], A[1], 0.5) # 第二节课 - 学习
set_pi(Pi, S[2], A[4], 0.5) # 第二节课 - 退出学习
set_pi(Pi, S[3], A[1], 0.5) # 第三节课 - 学习
set_pi(Pi, S[3], A[3], 0.5) # 第三节课 - 泡吧

print("----策略概率字典（矩阵）信息:----")
display_dict(Pi)
# 初始时价值为空，访问时会返回0
print("----状态转移概率字典（矩阵）信息:----")
V = {}
display_dict(V)

----策略概率字典（矩阵）信息:----
浏览手机中_浏览手机:　0.50
浏览手机中_离开浏览:　0.50
第一节课_浏览手机:　0.50
第一节课_学习:　0.50
第二节课_学习:　0.50
第二节课_退出学习:　0.50
第三节课_学习:　0.50
第三节课_泡吧:　0.50

----状态转移概率字典（矩阵）信息:----



#### 下面给出行为价值函数q(s,a)以及价值函数v_pi(s)的计算方法

In [5]:
def compute_q(MDP, V, s, a):
    '''
    根据给定的MDP，价值函数V，计算行为价值函数q(s,a)
    '''
    S, A, R, P, gamma = MDP
    q_sa = 0
    for s_prime in S:
        q_sa += get_prob(P, s, a, s_prime) * get_value(V, s_prime)
    q_sa = get_reward(R, s, a) + gamma * q_sa
    return q_sa


def compute_v(MDP, V, Pi, s):
    '''
    给定MDP下依据某一策略Pi和当前状态价值函数V计算价值函数v_pi_s
    '''
    S, A, R, P, gamma = MDP
    v_s = 0
    for a in A:
        v_s += get_pi(Pi, s, a) * compute_q(MDP, V, s, a)
    return v_s

#### 对于贝尔曼最优方程的理解，编程实现需要用到迭代方面的知识，我们暂时不考虑
